________
A notebook for visualizing NWP data and solar PV forecasts. 
________

In [26]:
import pandas as pd
import numpy as np

In [2]:
house_M = pd.read_excel("house M 5 min production.xlsx")
melania = pd.read_excel("melania 5 min production.xlsx")

In [3]:
START_DATE = '2020-05-05 00:00'
END_DATE = '2020-11-05 00:00'

In [4]:
# Apply rolling average and pick data from the end of every hour
# Select a time interval

def rolling_average_per_hour_interval(df):
    
    df['UTC'] = pd.to_datetime(df['UTC'])

    df_rolling_mean = df.loc[:, ['UTC', 'power [W]']].set_index('UTC').rolling('1h', min_periods = 1).mean()
    df_rolling_mean['UTC'] = df_rolling_mean.index
    df_rolling_mean = df_rolling_mean.reset_index(drop=True)
               
    for i in range(df_rolling_mean.shape[0]):
        if df_rolling_mean.UTC[i].hour == 1 and df_rolling_mean.UTC[i].minute == df_rolling_mean.UTC[i].second == 0:
            index_start = i
            break
            
                
    for i in range(df_rolling_mean.shape[0]-1, -1, -1):
        if df_rolling_mean.UTC[i].hour == df_rolling_mean.UTC[i].minute == df_rolling_mean.UTC[i].second == 0:
            index_stop = i
            break
            
    return df_rolling_mean.iloc[index_start:(index_stop+1)][::12].reset_index(drop = True)

In [5]:
house_M_hourly = rolling_average_per_hour_interval(house_M)
melania_hourly = rolling_average_per_hour_interval(melania)

In [6]:
import datetime
from suntime import Sun

def get_sunrise_sunset(df, latitude, longitude):
    
    period_start = df.loc[:, 'UTC'].iloc[0]
    year_start, month_start, day_start = period_start.year, period_start.month, period_start.day

    period_end = df.loc[:, 'UTC'].iloc[df.shape[0]-1]
    year_end, month_end, day_end = period_end.year, period_end.month, period_end.day
    
    all_timestamps_utc = pd.date_range(start = datetime.date(year_start, month_start, day_start), 
                                   end = datetime.date(year_end, month_end, day_end), freq="1D")
    
    sun = Sun(latitude, longitude)
    sunrises_sunsets = dict()
    
    for day in all_timestamps_utc:
        sunrises_sunsets[day.strftime('%Y-%m-%d %H:%M:%S').split()[0]] = {
                                                                            'sunrise': sun.get_sunrise_time(day).strftime('%Y-%m-%d %H:%M:%S'),
                                                                            'sunset': sun.get_sunset_time(day).strftime('%Y-%m-%d %H:%M:%S')
                                                                        }

    return sunrises_sunsets

In [7]:
latitude_Vuorela, longitude_Vuorela = 62.97999, 27.64920
sunrises_sunsets_Vuorela = get_sunrise_sunset(house_M_hourly, latitude_Vuorela, longitude_Vuorela)

latitude_Savilahti, longitude_Savilahti = 62.89216, 27.63362
sunrises_sunsets_Savilahti = get_sunrise_sunset(melania_hourly, latitude_Savilahti, longitude_Savilahti)

In [8]:
def check_sun(x, sunrises_sunsets):
    '''
    Check if a timestamp between sunrise and sunset
    '''
    if (x >= pd.to_datetime(sunrises_sunsets[x.strftime('%Y-%m-%d %H:%M:%S').split()[0]]['sunrise']) and 
        x <= pd.to_datetime(sunrises_sunsets[x.strftime('%Y-%m-%d %H:%M:%S').split()[0]]['sunset'])):
        return 1
    else:
        return 0

def add_zeros_night(df, sunrise_sunset, col):
    
    df['sun is up'] = df.UTC.apply(lambda x: check_sun(x, sunrise_sunset))
    df.loc[df[col].isna() & (df['sun is up'] == 0), col] = 0

In [9]:
add_zeros_night(house_M_hourly, sunrises_sunsets_Vuorela, 'power [W]')
add_zeros_night(melania_hourly, sunrises_sunsets_Savilahti, 'power [W]')

____________________________________
Read forecast from FMI for Vuorela
____________________________________

In [10]:
import sys
import glob

# Bank 1
pv_forecast_vuorela_1 = pd.DataFrame()

for file in glob.glob("./Vuorela_1/*.csv"):
    pv_forecast = pd.read_csv(file, infer_datetime_format=True, parse_dates=["time"])
    if list(pv_forecast['time'].dt.hour.values) == [20, 21, 22, 23,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                                                   13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0]:
        mask = [False]*5 + [True]*24
        
        pv_forecast_vuorela_1 = pd.concat([pv_forecast_vuorela_1, pv_forecast.loc[mask, :]], ignore_index=True)
        
    else:
        print("Diffent hours in the file:")
        print(file)

In [11]:
# Bank 2
pv_forecast_vuorela_2 = pd.DataFrame()

for file in glob.glob("./Vuorela_2/*.csv"):
    pv_forecast = pd.read_csv(file, infer_datetime_format=True, parse_dates=["time"])
    if list(pv_forecast['time'].dt.hour.values) == [20, 21, 22, 23,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                                                   13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0]:
        mask = [False]*5 + [True]*24
        
        pv_forecast_vuorela_2 = pd.concat([pv_forecast_vuorela_2, pv_forecast.loc[mask, :]], ignore_index=True)
        
    else:
        print("Diffent hours in the file:")
        print(file)

In [12]:
# Combine two banks
pv_forecast_vuorela = pv_forecast_vuorela_1.copy()
pv_forecast_vuorela['UTC'] = pv_forecast_vuorela_1['time']
pv_forecast_vuorela['forecast fmi [W]'] = pv_forecast_vuorela_1['pv_po'] + pv_forecast_vuorela_2['pv_po']
pv_forecast_vuorela.reset_index(drop = True, inplace = True)

____________________________________
Read forecast from FMI for Savilahti
____________________________________

In [13]:
import sys
import glob

pv_forecast_savilahti = pd.DataFrame()

for file in glob.glob("./Savilahti/*.csv"):
    pv_forecast = pd.read_csv(file, infer_datetime_format=True, parse_dates=["time"])
    if list(pv_forecast['time'].dt.hour.values) == [20, 21, 22, 23,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                                                   13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0]:
        mask = [False]*5 + [True]*24
        
        pv_forecast_savilahti = pd.concat([pv_forecast_savilahti, pv_forecast.loc[mask, :]], ignore_index=True)
        
    else:
        print("Diffent hours in the file:")
        print(file)

In [14]:
pv_forecast_savilahti['UTC'] = pv_forecast_savilahti['time']
pv_forecast_savilahti['forecast fmi [W]'] = pv_forecast_savilahti['pv_po']

In [16]:
latitude_Vuorela, longitude_Vuorela = 62.97999, 27.64920
sunrises_sunsets_Vuorela = get_sunrise_sunset(pv_forecast_vuorela, latitude_Vuorela, longitude_Vuorela)

latitude_Savilahti, longitude_Savilahti = 62.89216, 27.63362
sunrises_sunsets_Savilahti = get_sunrise_sunset(pv_forecast_savilahti, latitude_Savilahti, longitude_Savilahti)

In [17]:
add_zeros_night(pv_forecast_vuorela, sunrises_sunsets_Vuorela, 'forecast fmi [W]')
add_zeros_night(pv_forecast_savilahti, sunrises_sunsets_Savilahti, 'forecast fmi [W]')

In [19]:
pv_forecast_vuorela['wind_dir_cos'] = np.cos(pv_forecast_vuorela['wind_dir']*2*np.pi/360)
pv_forecast_vuorela['wind_dir_sin'] = np.sin(pv_forecast_vuorela['wind_dir']*2*np.pi/360)

pv_forecast_savilahti['wind_dir_cos'] = np.cos(pv_forecast_savilahti['wind_dir']*2*np.pi/360)
pv_forecast_savilahti['wind_dir_sin'] = np.sin(pv_forecast_savilahti['wind_dir']*2*np.pi/360)

In [20]:
house_M_hourly = pd.concat([pv_forecast_vuorela.set_index('UTC'),
                            house_M_hourly.set_index('UTC')], axis=1).reset_index(drop=False)
melania_hourly = pd.concat([pv_forecast_savilahti.set_index('UTC'),
                            melania_hourly.set_index('UTC')], axis=1).reset_index(drop=False)

In [21]:
class Site:
    def __init__(self, **kwargs):
        self.__name = kwargs.get('name', [])
        self.__dates = kwargs.get('dates', [])
        self.__power_measured = kwargs.get('power_measured', [])
        self.__power_forecast_fmi = kwargs.get('power_forecast_fmi', [])
        self.__predictors_forecast_fmi = kwargs.get('predictors_forecast_fmi', pd.DataFrame())
        #self.__power_predicted = None
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        self.__name = name
        
    @property
    def dates(self):
        return self.__dates
    
    @dates.setter
    def dates(self, dates):
        self.__dates = dates
    
    @property
    def power_measured(self):
        return self.__power_measured
    
    @power_measured.setter
    def power_measured(self, power_measured):
        self.__power_measured = power_measured   
    
    @property
    def power_forecast_fmi(self):
        return self.__power_forecast_fmi
    
    @power_forecast_fmi.setter
    def power_forecast_fmi(self, power_forecast_fmi):
        self.__power_forecast_fmi = power_forecast_fmi
        
    @property
    def predictors_forecast_fmi(self):
        return self.__predictors_forecast_fmi
    
    @predictors_forecast_fmi.setter
    def predictors_forecast_fmi(self, predictors_forecast_fmi):
        self.__predictors_forecast_fmi = predictors_forecast_fmi

In [22]:
class SiteInBokeh:
    def __init__(self):
        pass
    
    def name(self, Site):
        return Site.name
    
    def source_outputs(self, Site):
        return ColumnDataSource(data = {
                                    'UTC': Site.dates,
                                    'Measured [W]': Site.power_measured,
                                    'Forecast FMI [W]': Site.power_forecast_fmi,
                                  })
    
    def source_predictors(self, Site):
        return ColumnDataSource(data = Site.predictors_forecast_fmi)
    
    def x_range(self, Site):
        return (pd.to_datetime(Site.dates[0], format = '%Y-%m-%d'), 
               pd.to_datetime(Site.dates[len(Site.dates)-1], format = '%Y-%m-%d'))

    def data_table(self, Site):
        columns = [
                    TableColumn(field='UTC', title='UTC', formatter=DateFormatter(format="%m/%d/%Y %H:%M")),
                    TableColumn(field='Measured [W]', title='Measured [W]', formatter=NumberFormatter(format="0,0.00")),
                    TableColumn(field='Forecast FMI [W]', title='Forecast FMI [W]', formatter=NumberFormatter(format="0,0.00"))
                  ]
        return DataTable(source=self.source_outputs(Site), columns = columns, width=500, height = 300)

    def source_group(self, Site, group):
        df = pd.DataFrame({
                            'UTC': Site.dates,
                            'Measured [W]': Site.power_measured,
                            'Forecast FMI [W]': Site.power_forecast_fmi,
                           })

        df = pd.melt(df, id_vars=['UTC'], value_vars=['Measured [W]', 'Forecast FMI [W]'], var_name='group', value_name='Power [W]')

        df_group = df.loc[df.group == group]
        return ColumnDataSource(data = df_group)

In [23]:
predictors = ['UTC', 's_glob', 's_dif', 'T_AVG', 'wind_avg', 'wind_dir_cos', 'wind_dir_sin', 'albedo', 'prec_amt', 'cl_tot', 'cl_low', 'cl_med', 'cl_high']

vuorela_site = Site(name = 'Vuorela (a private house)',
                   dates=house_M_hourly['UTC'].values,
                   power_measured=house_M_hourly['power [W]'].values,
                   power_forecast_fmi=house_M_hourly['forecast fmi [W]'].values,
                   predictors_forecast_fmi = pv_forecast_vuorela.loc[:, predictors])

savilahti_site = Site(name = 'Savilahti (Melania)',
                   dates=melania_hourly['UTC'].values,
                   power_measured=melania_hourly['power [W]'].values,
                   power_forecast_fmi=melania_hourly['forecast fmi [W]'].values,
                   predictors_forecast_fmi = pv_forecast_savilahti.loc[:, predictors])

In [24]:
from bokeh.layouts import gridplot

def build_line(source_predictors, predictor):
    
    p = figure(plot_width=350, plot_height=300, x_axis_type='datetime', title = predictor,
                    tools = 'reset,wheel_zoom,box_zoom,pan,save')
        
    p.line(x='UTC', y=predictor, source=source_predictors, line_width=2, color="gray", alpha=0.75)

    hover_tool = HoverTool(tooltips=[('UTC', '@UTC{%Y-%m-%d %H:%M}'), (predictor, '@{0}'.format(predictor))],
                           formatters={'@UTC': 'datetime'},)
    p.add_tools(hover_tool)
    box = BoxAnnotation(left=datetime.datetime(2020, 5, 5), right = datetime.datetime(2020, 11, 5), fill_alpha=0.1, fill_color='green')
    p.add_layout(box)
    
    
    return p
        

def build_predictors(source_predictors, predictors, ncols):
   
    p = []
    for predictor in predictors[1:]:
        p.append(build_line(source_predictors, predictor))
        
    return gridplot(p, ncols=ncols)
        
    
    

In [25]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import FixedTicker, HoverTool, DataTable
from bokeh.layouts import layout
from bokeh.models import ColumnDataSource, TableColumn, NumberFormatter, DateFormatter, BoxAnnotation
import datetime

def build_dashboard(sites, predictors):
    
    tabs = []
    
    for site in sites:
    
        site_in_bokeh = SiteInBokeh()
        source_outputs = site_in_bokeh.source_outputs(site)
        data_table = site_in_bokeh.data_table(site)
        source_group_1 = site_in_bokeh.source_group(site, 'Measured [W]')
        source_group_2 = site_in_bokeh.source_group(site, 'Forecast FMI [W]')
        x_range = site_in_bokeh.x_range(site)
        source_predictors = site_in_bokeh.source_predictors(site)
        
        p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime', title = 'Power [W]',
                    x_range = x_range, tools = 'reset,wheel_zoom,box_zoom,pan,save')
        
        p.line(x='UTC', y='Power [W]', source=source_group_1, line_width=2, color="brown", legend_label='Measured', alpha=0.25)
        p.line(x='UTC', y='Power [W]', source=source_group_2, line_width=2, color="navy", legend_label='Forecast FMI', alpha=0.25)

        hover_tool = HoverTool(tooltips=[('Name','@group'), ('UTC', '@UTC{%Y-%m-%d %H:%M}'), ('Power [W]', '@{Power [W]}{0,0.00}')],
                               formatters={'@UTC': 'datetime'},)
        p.add_tools(hover_tool)
        box = BoxAnnotation(left=datetime.datetime(2020, 5, 5), right = datetime.datetime(2020, 11, 5), fill_alpha=0.1, fill_color='green')
        p.add_layout(box)
        
        ncols = 4
        predictors_plots = build_predictors(source_predictors, predictors, ncols)
        
        tab = Panel(child=layout([[p, data_table], [predictors_plots]]), title=site.name)
        tabs.append(tab)
        
    return Tabs(tabs=tabs)

output_file("measured_power_vs_forecast_fmi_plus_predictors.html", mode='inline')
show(build_dashboard([vuorela_site, savilahti_site], predictors))